In [5]:
from __future__ import print_function
import torch.utils.data as data
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn
from PIL import Image
import os
import os.path
import errno
import torch
import codecs

In [6]:
def get_int(b):
    return int(codecs.encode(b, 'hex'), 16)


def parse_byte(b):
    if isinstance(b, str):
        return ord(b)
    return b


def read_label_file(path):
    print("label Path", path)
    with open(path, 'rb') as f:
        data = f.read()
        assert get_int(data[:4]) == 2049
        length = get_int(data[4:8])
        labels = [parse_byte(b) for b in data[8:]]
        assert len(labels) == length
        print("Reading labels complete")
        return torch.LongTensor(labels)


def read_image_file(path):
    print("data Path", path)
    with open(path, 'rb') as f:
        data = f.read()
        assert get_int(data[:4]) == 2051
        length = get_int(data[4:8])
        num_rows = get_int(data[8:12])
        num_cols = get_int(data[12:16])
        images = []
        idx = 16
        for l in range(length):
            img = []
            images.append(img)
            for r in range(num_rows):
                row = []
                img.append(row)
                for c in range(num_cols):
                    row.append(parse_byte(data[idx]))
                    idx += 1
        assert len(images) == length
        print("Reading data complete")
        return torch.ByteTensor(images).view(-1, 28, 28)

I assume that the dataset is availabe in folder called 'raw'

Give full path to the folder. In my system the data is given in the path:'/home/akhilz/academics/2sem/pytorch/raw/'
Download data from github and after extracting put it in the folder 'raw' and change the path in below two cells.

In [7]:
training_set= (read_image_file('/home/akhilz/academics/2sem/pytorch/raw/train-images-idx3-ubyte'),
                read_label_file('/home/akhilz/academics/2sem/pytorch/raw/train-labels-idx1-ubyte'
                                ))

data Path /home/akhilz/academics/2sem/pytorch/raw/train-images-idx3-ubyte
Reading data complete
label Path /home/akhilz/academics/2sem/pytorch/raw/train-labels-idx1-ubyte
Reading labels complete


In [8]:
test_set= (read_image_file('/home/akhilz/academics/2sem/pytorch/raw/test-images-idx3-ubyte'),
          read_label_file('/home/akhilz/academics/2sem/pytorch/raw/test-labels-idx1-ubyte'))

data Path /home/akhilz/academics/2sem/pytorch/raw/test-images-idx3-ubyte
Reading data complete
label Path /home/akhilz/academics/2sem/pytorch/raw/test-labels-idx1-ubyte
Reading labels complete


In [9]:
with open('./data/processed/training.pt','wb') as f:
    torch.save(training_set,f)

with open('./data/processed/test.pt','wb') as f:
    torch.save(test_set,f)

In [10]:
class fashion(data.Dataset):
    raw_folder= 'raw'
    processed_folder= 'processed'
    training_file= 'training.pt'
    test_file= 'test.pt'
    
    def __init__(self, root, train= True, transform= None, target_transform= None, download= False):
        # if root= '~/folder_name/' os.path.expanduser(root) converts to /home/folder_name
        self.root= os.path.expanduser(root) 
        self.transform= transform
        self.target_transform= target_transform
        self.train= train
    
        if not self.__check_exists():
            raise RuntimeError('Dataset not found.')
    
        if self.train:
            self.train_data, self.train_labels = torch.load(os.path.join(root, self.processed_folder, self.training_file))
        else:
            self.test_data, self.test_labels = torch.load(os.path.join(root, self.processed_folder, self.test_file))
        
    def __getitem__(self, index):
        if self.train:
            img, target= self.train_data[index], self.train_labels[index]
        else:
            img, target= self.test_data[index], self.test_labels[index]
        
        #read about PIL
        #It is done to make this module consistent with all other datasets say RGB etc.
        img= Image.fromarray(img.numpy(), mode='L')
        
        if self.transform:
            img= self.transform(img)
        
        if self.target_transform:
            target= self.target_transform(target)
        
        return img, target
    
    def __len__(self):
        if self.train:
            return len(self.train_data)
        return len(self.test_data)
    
    def __check_exists(self):
        print(os.path.join(self.root, self.processed_folder, self.test_file))
        return os.path.exists(os.path.join(self.root, self.processed_folder, self.test_file)) and\
                os.path.exists(os.path.join(self.root, self.processed_folder, self.test_file))
        
    def get_int(self,b):
        return int(codecs.encode(b, 'hex'), 16)


    def parse_byte(self,b):
        if isinstance(b, str):
            return ord(b)
        return b


    def read_label_file(self,path):
        with open(path, 'rb') as f:
            data = f.read()
            assert self.get_int(data[:4]) == 2049
            length = self.get_int(data[4:8])
            labels = [self.parse_byte(b) for b in data[8:]]
            assert len(labels) == length
            return torch.LongTensor(labels)
    
    def read_image_file(self,path):
        with open(path, 'rb') as f:
            data = f.read()
            assert self.get_int(data[:4]) == 2051
            length = self.get_int(data[4:8])
            num_rows = self.get_int(data[8:12])
            num_cols = self.get_int(data[12:16])
            images = []
            idx = 16
            for l in range(length):
                img = []
                images.append(img)
                for r in range(num_rows):
                    row = []
                    img.append(row)
                    for c in range(num_cols):
                        row.append(self.parse_byte(data[idx]))
                        idx += 1
            assert len(images) == length
            return torch.ByteTensor(images).view(-1, 28, 28)


#### Step 1
Loading Dataset

In [11]:
normalize= transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]],
                               std= [x/255.0 for x in [63.0, 62.1, 66.7]])
transform= transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.137,),(0.3081,))])

In [12]:
train_dataset= fashion(root='/home/akhilz/academics/2sem/pytorch/',
                       train= True, transform= transform)
test_dataset= fashion(root='/home/akhilz/academics/2sem/pytorch/', 
                      train= False, transform= transform)

/home/akhilz/academics/2sem/pytorch/processed/test.pt
/home/akhilz/academics/2sem/pytorch/processed/test.pt


#### Step 2

Making dataset iterable

One Epoch is when an ENTIRE dataset is passed forward and backward through the neural network only ONCE.

Batch_size: Total number of training examples present in a single batch.

Iterations is the number of batches needed to complete one epoch.

The number of batches is equal to number of iterations for one epoch.

How to calculate batch_size,num_epochs, n_iters given dataset size?

Example: We can divide the dataset of 2000 examples into batches of 500 then it will take 4 iterations to complete 1 epoch.


In [24]:
batch_size= 256
n_iters= 4000
num_epochs= n_iters /(len(train_dataset) / batch_size)
num_epochs= int(num_epochs)

train_loader= torch.utils.data.DataLoader(dataset= train_dataset,
                                         batch_size= batch_size,
                                         shuffle= False)
test_loader=  torch.utils.data.DataLoader(dataset= test_dataset,
                                         batch_size= batch_size,
                                         shuffle= False)

#### Step 3

Creating a model (CNN)

In [25]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.cnn1= nn.Conv2d(#input shape(1,28,28)
            in_channels=1, #input height
            out_channels= 16, #n_filters
            kernel_size= 5, #filter size
            stride= 1, #filter movement step
            padding= 0) # if want same width and length of this image after con2d, padding=(kernel_size-1)/2 if stride=1
            # output shape(16,24,24)
        self.bc1= nn.BatchNorm2d(16)
        self.relu1= nn.ReLU() #activation
        
        # MAx pool 1
        self.maxpool1= nn.MaxPool2d(kernel_size= 2)  # choose max value in 2x2 area, output shape (16, 12, 12)
        
        
        # Convolution 2
        self.cnn2= nn.Conv2d( #input size (16,12,12)
            in_channels= 16, 
            out_channels= 32, 
            kernel_size= 5, 
            stride= 1, 
            padding= 0) #output(32,8,8)
        self.bc2= nn.BatchNorm2d(32)
        self.relu2= nn.ReLU()
        #MAx pool 2
        self.maxpool2= nn.MaxPool2d(kernel_size= 2)#output size(32, 10, 10)
        
        self.dropout= nn.Dropout(p= 0.5)
        
        #Fully connected 1
        self.fc1= nn.Linear(32*4*4,10)
        
    def forward(self, x):
        #Convolution 1
        out= self.cnn1(x)
        out= self.bc1(out)
        out= self.relu1(out)
        out= self.maxpool1(out)
        
        #convolution 2
        
        out= self.cnn2(out)
        out= self.bc2(out)
        out= self.relu2(out)
        out= self.maxpool2(out)
        
        #after maxpool2 the output size is(100, 32, 4, 4)
        out= out.view(out.size(0), -1)
        #now the output size is(100, 32*4*4)
        
        out= self.dropout(out)
        
        #apply fully connected layer(linear function)
        out= self.fc1(out)
        
        return out

#### Step 4


In [26]:
#Instantiate model class       
model= CNNModel()


To print the architecure of your model just do the following. Check if it gives what you expected.

In [27]:
print(model)

CNNModel (
  (cnn1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
  (bc1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
  (relu1): ReLU ()
  (maxpool1): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (cnn2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (bc2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
  (relu2): ReLU ()
  (maxpool2): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (dropout): Dropout (p = 0.5)
  (fc1): Linear (512 -> 10)
)


#### Step 5

Instantiate Loss class and Optimizer class

In [28]:
#Instantiate Loss class
criterion= nn.CrossEntropyLoss()

#Instantiate optimizer class
learning_rate= 0.01
optimizer= torch.optim.Adam(model.parameters(), lr=learning_rate)

#### Step 6

We are half way done.  
Now we are ready to train the model. 

In [29]:
iter= 0
for epoch in range(num_epochs):
    for i, (images,labels) in enumerate(train_loader):
        images= Variable(images)
        labels= Variable(labels)
        
        #clear gradient w.r.t. parameters
        optimizer.zero_grad()
        
        #Forward pass to get output/logits
        outputs= model(images)
        
        #Calculates the loss :  we are using softmax cross entropy loss
        loss= criterion(outputs, labels)
        
        #Getting gradient w.r.t. parameters
        loss.backward()
        
        #update parametrs
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            #calculate accuracy
            correct= 0
            total= 0
            
            #Iterate through test dataset
            for images, labels in test_loader:
                images= Variable(images)
                
                #Forward pass to get the output/logits
                outputs= model(images)
                
                #Get prediction from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                #Total number of labels
                total += labels.size(0)
                
                correct += (predicted == labels).sum()
                
            accuracy= 100 * correct / total
            
            #print loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))
            

Iteration: 500. Loss: 0.3766932785511017. Accuracy: 85.95
Iteration: 1000. Loss: 0.34714993834495544. Accuracy: 87.27
Iteration: 1500. Loss: 0.3242199420928955. Accuracy: 87.54
Iteration: 2000. Loss: 0.28273841738700867. Accuracy: 87.59
Iteration: 2500. Loss: 0.24641162157058716. Accuracy: 87.92
Iteration: 3000. Loss: 0.2886813282966614. Accuracy: 88.29
Iteration: 3500. Loss: 0.3215324878692627. Accuracy: 88.06


#### Save the model


In [ ]:
save_model= True
if save_model is True:
    #save only parameters
    torch.save(model.state_dict(),'fashion_model_trained.pk1')